In [ ]:
import cv2
import os
import subprocess
import tempfile
import shutil

import pandas as pd

from tqdm.auto import tqdm

Code for video annotation from @samhuddleston https://www.kaggle.com/samhuddleston/nfl-1st-and-future-getting-started kernel

In [ ]:
def annotate_video(
        video_path: str,
        video_labels: pd.DataFrame,
        output_path: str,
        convert: bool = False) -> str:
    '''
    Create a function to annotate the video at the provided path using labels
    from the provided dataframe, return the path of the video
    '''

    VIDEO_CODEC = "MP4V"
    HELMET_COLOR = (0, 0, 0)    # Black
    IMPACT_COLOR = (0, 0, 255)  # Red
    video_name = os.path.basename(video_path)

    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    with tempfile.TemporaryDirectory() as tmp_dir:
        tmp_output_path = os.path.join(tmp_dir, video_name)
        output_video = cv2.VideoWriter(tmp_output_path,
                                       cv2.VideoWriter_fourcc(*VIDEO_CODEC),
                                       fps, (width, height))
        frame = 0
        while True:
            it_worked, img = vidcap.read()
            if not it_worked:
                break

            # We need to add 1 to the frame count to match the label frame index
            # that starts at 1
            frame += 1

            # Let's add a frame index to the video so we can track where we are
            img_name = f"{video_name}_frame{frame}"
            cv2.putText(img, img_name, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                        HELMET_COLOR, thickness=2)

            # Now, add the boxes
            boxes = video_labels.query("video == @video_name and frame == @frame")
            for box in boxes.itertuples(index=False):
                # Filter for definitive head impacts and turn labels red
                if box.impact == 1 and box.confidence > 1 and box.visibility > 0:
                    color, thickness = IMPACT_COLOR, 2
                else:
                    color, thickness = HELMET_COLOR, 1
                # Add a box around the helmet
                cv2.rectangle(img, (box.left, box.top),
                            (box.left + box.width, box.top + box.height),
                            color, thickness=thickness)
                cv2.putText(img, box.label, (box.left, max(0, box.top - 5)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, thickness=1)
            output_video.write(img)
        output_video.release()

        if os.path.exists(output_path):
            os.remove(output_path)
        # Not all browsers support the codec, we will re-load the file at
        # tmp_output_path and convert to a codec that is more broadly
        # readable using ffmpeg
        if convert:
            subprocess.run(["ffmpeg", "-i", tmp_output_path,
                            "-crf", "18", "-preset", "veryfast",
                            "-vcodec", "libx264", output_path])
        else:
            shutil.copy(tmp_output_path, output_path)

    return output_path

In [ ]:
# Define input and output paths
data_path = '/kaggle/input/nfl-impact-detection'
dest_video_path = './output/annotated_videos/'

video_path = os.path.join(data_path, 'train')
video_files = os.listdir(video_path)
video_labels = pd.read_csv(os.path.join(data_path, 'train_labels.csv'))

os.makedirs(dest_video_path, exist_ok=True)

for video_file in tqdm(video_files):
    annotate_video(os.path.join(video_path, video_file), video_labels,
                   os.path.join(dest_video_path, video_file),
                   True)